In [1]:
from app import commit_lux_to_db, db, Campaign
import datetime as dt

## Initial db creation 

In [2]:
db.create_all()

In [3]:
t = dt.datetime.now()    
next_hour = t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1)
hourly = Campaign(id=0, title="hourly", start=next_hour, end=dt.datetime(2099,1,1), interval=dt.timedelta(seconds=3600))
db.session.add(hourly)
db.session.commit()
hourly

Campaign(id:0, 'hourly' is active:True, start:2021-12-05 13:00:00, end:2099-01-01 00:00:00, interval:1:00:00)

## query

In [2]:
Campaign.query.all()

[Campaign(id:0, 'hourly' is active:True, start:2021-12-05 13:00:00, end:2099-01-01 00:00:00, interval:1:00:00),
 Campaign(id:1, 'understand the light' is active:True, start:2021-12-05 12:54:00, end:2021-12-12 12:54:00, interval:0:00:30)]

In [3]:
Campaign.query.filter(Campaign.is_active==True).all()

[Campaign(id:0, 'hourly' is active:True, start:2021-12-05 13:00:00, end:2099-01-01 00:00:00, interval:1:00:00),
 Campaign(id:1, 'understand the light' is active:True, start:2021-12-05 12:54:00, end:2021-12-12 12:54:00, interval:0:00:30)]

In [5]:
now = dt.datetime.now()
cmps = Campaign.query.filter(Campaign.is_active==True).filter(Campaign.end > now).all()
cmps

[Campaign(id:0, 'hourly' is active:True, start:2021-12-05 13:00:00, end:2099-01-01 00:00:00, interval:1:00:00),
 Campaign(id:1, 'understand the light' is active:True, start:2021-12-05 12:54:00, end:2021-12-12 12:54:00, interval:0:00:30)]

In [6]:
cmps

[Campaign(id:0, 'hourly' is active:True, start:2021-12-05 13:00:00, end:2099-01-01 00:00:00, interval:1:00:00),
 Campaign(id:1, 'understand the light' is active:True, start:2021-12-05 12:54:00, end:2021-12-12 12:54:00, interval:0:00:30)]

In [94]:
def get_next_capture(cmps):
    """
    Returns <dict> of for the next capture, with two items:
        "sleeptime": time <float> in seconds to sleep until this capture
        "campain": campaign <Campaign> of the next captur

    Args:
        cmps <list>: List of campaigns <Campaign>
    """
    pending = {}   # pending captures with <timedelta> keys and <int> index values
    now = dt.datetime.now()
    for idx, cmp in enumerate(cmps):
        # Overdue detection for immidiate capture
        overdue = False
        if now > cmp.start:
            if cmp.previous is None:
                overdue = True
            else:
                next = cmp.previous + cmp.interval
                print("next:", next)
                if now >= next:
                    overdue = True
                    
        if overdue:
            return {"sleeptime": 0, "campaign": cmp}
        
        if now <= cmp.start:  # case: start time in the future
            next_capture = cmp.start
        else:  # case: ongoing campaign
            next_capture = cmp.previous + cmp.interval
        seconds_until_capture = max((next_capture - now) / dt.timedelta(seconds=1.0), 0.0) 
        print("next_capture:", next_capture, seconds_until_capture)
        pending[seconds_until_capture] = cmp
            
    print("todo: realign seconds function")

    print("pending:")
    for item in pending.items():
        print("-", item)
    min_sleeptime, next_cmp = sorted(pending.items())[0]
    return {"sleeptime": min_sleeptime, "campaign": next_cmp}
    


In [95]:
get_next_capture(cmps)

{'sleeptime': 0,
 'campaign': Campaign(id:0, 'hourly' is active:True, start:2021-12-05 13:00:00, end:2099-01-01 00:00:00, interval:1:00:00)}

In [96]:
testvector = [Campaign(title="T1", is_active=True, start=dt.datetime(2021,12,6), end=dt.datetime(2099,1,1), previous=None, interval=dt.timedelta(seconds=3600)), 
              Campaign(title="T2", is_active=True, start=dt.datetime(2021,12,5), end=dt.datetime(2099,1,1), previous=dt.datetime(2021,12,5,21), interval=dt.timedelta(seconds=3600)), 
              Campaign(title="T3", is_active=True, start=dt.datetime(2021,12,8), end=dt.datetime(2099,1,1), previous=None, interval=dt.timedelta(seconds=3600)), 
              ]

In [97]:
get_next_capture(testvector)

next_capture: 2021-12-06 00:00:00 9199.99196
next: 2021-12-05 22:00:00
next_capture: 2021-12-05 22:00:00 1999.99196
next_capture: 2021-12-08 00:00:00 181999.99196
todo: realign seconds function
pending:
- (9199.99196, Campaign(id:None, 'T1' is active:True, start:2021-12-06 00:00:00, end:2099-01-01 00:00:00, interval:1:00:00))
- (1999.99196, Campaign(id:None, 'T2' is active:True, start:2021-12-05 00:00:00, end:2099-01-01 00:00:00, interval:1:00:00))
- (181999.99196, Campaign(id:None, 'T3' is active:True, start:2021-12-08 00:00:00, end:2099-01-01 00:00:00, interval:1:00:00))


{'sleeptime': 1999.99196,
 'campaign': Campaign(id:None, 'T2' is active:True, start:2021-12-05 00:00:00, end:2099-01-01 00:00:00, interval:1:00:00)}